# Animal Agriculture Analysis: ETL Pipeline

## Data extraction

In [1]:
from owid.catalog import charts
import pandas as pd
import numpy as np
import pandas as pd
import requests
import matplotlib
from sqlalchemy import create_engine


In [2]:
# URL paths to relevant charts
milks_path = 'https://ourworldindata.org/grapher/environmental-footprint-milks'
feed_per_kg_path = 'https://ourworldindata.org/grapher/feed-required-to-produce-one-kilogram-of-meat-or-dairy-product'
total_slaughter_path = 'https://ourworldindata.org/grapher/animals-slaughtered-for-meat'
protein_efficiency_path = 'https://ourworldindata.org/grapher/protein-efficiency-of-meat-and-dairy-production'
meat_production_path = 'https://ourworldindata.org/grapher/global-meat-production-by-livestock-type'


In [3]:
# get the data for each chart by URL
milks_df = charts.get_data(milks_path)
feed_per_kg_df = charts.get_data(feed_per_kg_path)
total_slaughter_df = charts.get_data(total_slaughter_path)
protein_efficiency_df = charts.get_data(protein_efficiency_path)
meat_production_df = charts.get_data(meat_production_path)


## Data transformation

In [4]:
# Standardize column names

# Standardize milks columns
milks_renaming = {
    '_242004': 'land_use_m2',
    '_242005': 'ghg_co2eq_kg',
    '_242006': 'water_use_L',
    '_242007': 'eutrophication_PO4eq_g'
    }

# Standardize total slaughter columns
total_slaughter_renaming = {
    'meat_of_cattle_with_the_bone__fresh_or_chilled__00000867__producing_or_slaughtered_animals__005320__animals': 'cattle',
    'meat__goat__00001017__producing_or_slaughtered_animals__005320__animals': 'goat',
    'meat__chicken__00001058__producing_or_slaughtered_animals__005321__animals': 'chicken',
    'meat__turkey__00001080__producing_or_slaughtered_animals__005321__animals': 'turkey',
    'meat__pig__00001035__producing_or_slaughtered_animals__005320__animals': 'pig',
    'meat__lamb_and_mutton__00000977__producing_or_slaughtered_animals__005320__animals': 'sheep',
    'meat__duck__00001069__producing_or_slaughtered_animals__005321__animals': 'duck'
    }

# Standardize meat production columns
meat_production_renaming = {
    'meat__game__00001163__production__005510__tonnes': 'wild_hunt_tonnes',
    'meat__horse__00001097__production__005510__tonnes': 'horse_tonnes',
    'meat__camel__00001127__production__005510__tonnes': 'camel_tonnes',
    'meat__sheep_and_goat__00001807__production__005510__tonnes': 'sheep_and_goat_tonnes',
    'meat__beef_and_buffalo__00001806__production__005510__tonnes': 'beef_and_buffalo_tonnes',
    'meat__pig__00001035__production__005510__tonnes': 'pig_tonnes',
    'meat__poultry__00001808__production__005510__tonnes': 'poultry_tonnes'
    }


In [5]:
milks_df = milks_df.rename(columns=milks_renaming)
total_slaughter_df = total_slaughter_df.rename(columns=total_slaughter_renaming)
meat_production_df = meat_production_df.rename(columns=meat_production_renaming)


In [6]:
world_total_slaughter_df = total_slaughter_df.loc[total_slaughter_df['entities'] == 'World'].copy()
mexico_total_slaughter_df = total_slaughter_df.loc[total_slaughter_df['entities'] == 'Mexico'].copy()

# Drop redundant columns
world_total_slaughter_df = world_total_slaughter_df.drop(columns=['entities'])
mexico_total_slaughter_df = mexico_total_slaughter_df.drop(columns=['entities'])


In [7]:
world_meat_production_df = meat_production_df.loc[meat_production_df['entities'] == 'World'].copy()
mexico_meat_production_df = meat_production_df.loc[meat_production_df['entities'] == 'Mexico'].copy()

# Drop redundant columns
world_meat_production_df = world_meat_production_df.drop(columns=['entities'])
mexico_meat_production_df = mexico_meat_production_df.drop(columns=['entities'])


## Load

In [8]:
# Create the SQLite engine
engine = create_engine('sqlite:///animal_agriculture.db')


In [9]:
# Write each DataFrame to the database
world_total_slaughter_df.to_sql('world_total_slaughter', con=engine, if_exists='replace', index=False)
mexico_total_slaughter_df.to_sql('mexico_total_slaughter', con=engine, if_exists='replace', index=False)
world_meat_production_df.to_sql('world_meat_production', con=engine, if_exists='replace', index=False)
mexico_meat_production_df.to_sql('mexico_meat_production', con=engine, if_exists='replace', index=False)
milks_df.to_sql('milks_comparison', con=engine, if_exists='replace', index=False)


5

# Example Queries

In [10]:
# Example query: top years of beef production
query = """
SELECT years, beef_and_buffalo_tonnes
FROM world_meat_production
ORDER BY beef_and_buffalo_tonnes DESC
LIMIT 10
"""

df_top_beef = pd.read_sql(query, con=engine)
print(df_top_beef)

   years  beef_and_buffalo_tonnes
0   2023               76560810.0
1   2022               75423450.0
2   2021               74071690.0
3   2019               73993960.0
4   2020               73003380.0
5   2018               72966970.0
6   2017               71118024.0
7   2016               69892300.0
8   2014               69797700.0
9   2015               69547440.0


# Create CSV files for Power BI

In [15]:
name = "milks_comparison"

query = f"""
SELECT *
FROM {name}
"""

df = pd.read_sql(query, con=engine)
df.to_csv(f"{name}.csv", index=False)